<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF;">

# Exercise 1 : Visualizing connectivity across cell type and space

The depth of a neuron in cortex tells us a lot about its cell type and role in the cortical network.
Where a neuron makes synapses has important implications for its function, for example whether it targets cell bodies, basal dendrites, or apical tufts.

In this exercise, we'll learn how to plot these properties of neurons outputs using the data from V1DD and standard plotting tools in Matplotlib.

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF;">
We'll start by loading up much of the same data from Workshop 1.
</div>

In [ ]:
# Import packages
import sys
from os.path import join as pjoin
import platform

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

from standard_transform.datasets import v1dd_ds

# Add the directory for the data and utilities
mat_version = 1196

platstring = platform.platform()
system = platform.system()
if system == "Darwin":
    # macOS
    data_root = "/Volumes/Brain2025/"
elif system == "Windows":
    # Windows (replace with the drive letter of USB drive)
    data_root = "E:/"
elif "amzn" in platstring:
    # then on CodeOcean
    data_root = "/data/"
else:
    # then your own linux platform
    # EDIT location where you mounted hard drive
    data_root = "/media/$USERNAME/Brain2025/"

# Set the directory to load prepared data and utility code
data_dir = pjoin(data_root, f"v1dd_{mat_version}")
utils_dir = pjoin("..", "utils")

# Add utilities to path
sys.path.append(utils_dir)


In [ ]:
# load the cell type information
ct_df = pd.read_feather(f"{data_dir}/soma_and_cell_type_{mat_version}.feather")

# Load synapses 
syn_df = syn_df = pd.read_feather(f"{data_dir}/syn_df_all_to_proofread_to_all_{mat_version}.feather")

# Load the proofreading info
proofread_ids = np.load(pjoin(data_dir, f"proofread_axon_list_{mat_version}.npy"))

# These are approximate depth values in post-transformed microns for the pial surface, L1-L2/3 border, L2/3-L4 border, etc. down to the L6-wm border.
layer_bounds = [
    -15,
    91,
    261,
    391,
    537,
    753,
]  

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF;">

## Step 0. Prepare some useful dataframes

1. Set up a dataframe with cell types and soma depth as well as if cells are proofread or not.
2. Set up the synapse dataframe for querying later.

</div>

In [ ]:
ct_df["soma_depth"] = (
    ct_df["pt_position_trform_y"] / 1_000
)  # Convert from nanometers to microns
ct_df.soma_depth.head()  # Approximate cell depth from pial surface in microns

In [ ]:
# What we are about to do requres that no root ids are duplicated in this, so let's double check:

ct_df.drop_duplicates("pt_root_id", keep=False, inplace=True)

In [ ]:
# Add proofreading information
ct_df["is_proofread"] = ct_df["pt_root_id"].isin(proofread_ids)

In [ ]:
# Add the synapse depth in consistent coordinates

syn_df["synapse_depth"] = v1dd_ds.transform_nm.apply_dataframe(
    "ctr_pt_position", syn_df, projection="y"
)

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF;">

# Exercise 1 : Visualizing Connectivity across cell type and space

## Part 1:
We want to visualize the combination of the spatial distribution of a neuron's synaptic outputs and where on their target cells they connect.

Let's try to visualize these two aspects together — where a neuron makes its synapses and where the cell body is of the target neuron.

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<b> Task 1a: </b> For root id `864691132729533614`, a basket cell, make a scatter plot with the following requirements:

1. Every dot is a synapse from one cell of interest onto any cell-typed target cell.
2. The x-axis represents the depth of the target cell's *soma*.
3. The y-axis represents the depth of the *synapse*.
4. Each point is colored by the cell type of the target cell. It may be convenient to use two panels for extra clarity.

Tip: Start by making a dataframe where every row is a synapse, and each column contains all the relevant information (e.g., synapse depth, target cell depth, target cell type).

</div>



<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<b>Task 1b:</b> After you are happy with your plot for that cell, please look at a cell from another cell type, dendrite-targeting interneuron `864691132834855816`. What do you notice about the differences in how these plots appear? What can you infer about how these two different neurons connect to their targets?

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

<b> Task 1c: </b> Find and plot the outputs of an excitatory neuron. How, if at all, do they differ from these inhibitory neurons?

</div>

<div style="border-left: 3px solid #000; padding: 1px; padding-left: 10px; background: #F0FAFF;">

# Part 2:

Knowing that depth matters a great deal, let's now visualize how neurons of different depths connect to one another.
This will be like an adacency matrix, except instead of giving each connected pair of neurons a row and column, we will plot it at their depths.
One nice thing about this plot is that, while cell typing can sometimes yield borderline or unclear edge cases, the depth of a cell is concrete and we can infer useful information about the laminar organization of the cortical circuit directly.

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Task 2a:** Make a scatterplot where:
1. Each dot is a connection from one proofread presynaptic neuron to one proofread postsynaptic neuron.
2. The x-axis represents the depth of the presynaptic neuron's soma.
3. The y-axis represents the depth of the postsynaptic neuron's soma.
4. Points are sized by the strength of the synaptic connection, as measured by the summed synapse size.

As before, add layer boundaries for context.

**Question:** Do you think summed synapse size or synapse count is a better measure of connection strength? Why? Is this true for all connections?

</div>

<div style="background: #f6d5f2ff; border-radius: 3px; padding: 10px;">

Tip: Explore the size distribution of synaptic connections so you can best display their sizes in the scatterplot.

</div>

<div style="background: #DFF0D8; border-radius: 3px; padding: 10px;">

**Task 2b:** Next, add cell type information. Instead of one scatter plot, make four plots, where one plot corresponds to E $\rightarrow$ E connections, one for E $\rightarrow$ I connections, one for I $\rightarrow$ E connections, and one for I $\rightarrow$ I connections.

**Question:** What do you notice about the geometry of each type of connection and how information flows across layers?

If you want to see what, if anything, changes when including *all* postsynaptic neurons (instead of only proofread), feel free to do that as well as a stretch goal.

</div>